# gyst (get your sheets together!) : guide fields, beta, and magnetization

This notebook is part of gyst (get your sheets together!).

Here, we get the guide fields, plasma beta, and magnetization around prospective reconnection sites.

## 1) Setup

In [ ]:
%pylab inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import argparse
from scipy import interpolate
import tqdm
import nmmn.lsd, nmmn.misc
import os.path
import pandas as pd
import astropy.visualization

## 2) Get guide fields, beta, and magnetization

### 2.1) Decide if we want disc, outflows, or jet

In [ ]:
disc = 0
outflows = 0
jet = 1

### 2.2) Get quantities

In [ ]:
sigmaext = []
betaext = []
sigmaext_center = []
betaext_center = []
BrBphiext = []
BthBphiext = []
#Vrecext = []
#Vrecstdevext = []
                
dumpmin, dumpmax = 0, 600
#imin = 800
#imax = imin+400
for dumpnumber in range(dumpmin, dumpmax+1):
    
    Vrec_avg = 0.
    Vrec_count = 0
    Vrec = []
    
    for i in range(0,N1):
        for j in range(0,N2):
            for k in range(0,N3):
                if (disc):
                    myfilename = "/work/gustavo/gyst/sheets_disc/dump%03d_%d_%d_%d_s" %(dumpnumber,i,j,k)
                elif (outflows):
                    myfilename = "/work/gustavo/gyst/sheets_outflows/dump%03d_%d_%d_%d_s" %(dumpnumber,i,j,k)
                elif (jet):
                    myfilename = "/work/gustavo/gyst/sheets_jet/dump%03d_%d_%d_%d_s" %(dumpnumber,i,j,k)
                if (os.path.isfile(myfilename)):
                    #print(i, j, k)

                    myfile = open(myfilename, "rb")
                    dtype = np.float64
                    body = np.fromfile(myfile,dtype=dtype,count=-1)
                    ind2 = int((len(body))/10)
                    mydata = body.view().reshape((10,ind2), order='F')
                    
                    myBr       = mydata[0]
                    myBth      = mydata[1]
                    myBphi     = mydata[2]
                    myJ        = mydata[3]
                    mybeta     = mydata[4]
                    mysigma    = mydata[5]
                    mysigmaphi = mydata[6]
                    myi        = mydata[7]
                    myj        = mydata[8]
                    myk        = mydata[9]
                    myfile.close()

                    sigmaext.append(mysigma[0])
                    sigmaext.append(mysigma[-1])
                    betaext.append(mybeta[0])
                    betaext.append(mybeta[-1])
                    # central quantities
                    sigmaext_center.append(sigma[i][j][k])
                    betaext_center.append(betapl[i][j][k])
                    BrBphiext.append(abs(B[1][i][j][k]/B[3][i][j][k]))
                    BthBphiext.append(abs(B[2][i][j][k]/B[3][i][j][k]))
                    
BrBphiext = [x for x in BrBphiext if str(x) != 'nan']
BrBphiext = [x for x in BrBphiext if str(x) != 'inf']
BrBphiext = np.array(BrBphiext)

BthBphiext = [x for x in BthBphiext if str(x) != 'nan']
BthBphiext = [x for x in BthBphiext if str(x) != 'inf']
BthBphiext = np.array(BthBphiext)

betaext = [x for x in betaext if str(x) != 'nan']
betaext = [x for x in betaext if str(x) != 'inf']
betaext = np.array(betaext)

sigmaext = [x for x in sigmaext if str(x) != 'nan']
sigmaext = [x for x in sigmaext if str(x) != 'inf']
sigmaext = np.array(sigmaext)

betaext_center = [x for x in betaext_center if str(x) != 'nan']
betaext_center = [x for x in betaext_center if str(x) != 'inf']
betaext_center = np.array(betaext_center)

sigmaext_center = [x for x in sigmaext_center if str(x) != 'nan']
sigmaext_center = [x for x in sigmaext_center if str(x) != 'inf']
sigmaext_center = np.array(sigmaext_center)

#Vrecext = [x for x in Vrecext if str(x) != 'nan']
#Vrecext = [x for x in Vrecext if str(x) != 'inf']
#Vrecext = np.array(Vrecext)

#Vrecstdevext = [x for x in Vrecstdevext if str(x) != 'nan']
#Vrecstdevext = [x for x in Vrecstdevext if str(x) != 'inf']
#Vrecstdevext = np.array(Vrecstdevext)

### 2.3) Save quantities in files

In [ ]:
if (disc):
    guidefield_r_file = open("guidefield_r_2D_disc.dat","w+")
    guidefield_th_file = open("guidefield_th_2D_disc.dat","w+")
    sigma_file = open("sigma_2D_disc.dat","w+")
    beta_file = open("beta_2D_disc.dat","w+")
    #sigmacenter_file = open("sigma_center_2D_disc.dat","w+")
    #betacenter_file = open("beta_center_2D_disc.dat","w+")
    
elif (outflows):
    guidefield_r_file = open("guidefield_r_2D_outflows.dat","w+")
    guidefield_th_file = open("guidefield_th_2D_outflows.dat","w+")
    sigma_file = open("sigma_2D_outflows.dat","w+")
    beta_file = open("beta_2D_outflows.dat","w+")
    #sigmacenter_file = open("sigma_center_2D_jet.dat","w+")
    #betacenter_file = open("beta_center_2D_jet.dat","w+")

elif (jet):
    guidefield_r_file = open("guidefield_r_2D_jet.dat","w+")
    guidefield_th_file = open("guidefield_th_2D_jet.dat","w+")
    sigma_file = open("sigma_2D_jet.dat","w+")
    beta_file = open("beta_2D_jet.dat","w+")
    #sigmacenter_file = open("sigma_center_2D_jet.dat","w+")
    #betacenter_file = open("beta_center_2D_jet.dat","w+")

for line in range(len(BrBphiext)):
    guidefield_r_file.write(str(BrBphiext[line]))
    guidefield_r_file.write("\n")
guidefield_r_file.close()

for line in range(len(BthBphiext)):
    guidefield_th_file.write(str(BthBphiext[line]))
    guidefield_th_file.write("\n")
guidefield_th_file.close()

for line in range(len(sigmaext)):
    sigma_file.write(str(sigmaext[line]))
    sigma_file.write("\n")
sigma_file.close()

for line in range(len(betaext)):
    beta_file.write(str(betaext[line]))
    beta_file.write("\n")
beta_file.close()
"""
for line in range(len(sigmaext_center)):
    sigmacenter_file.write(str(sigmaext_center[line]))
    sigmacenter_file.write("\n")
sigmacenter_file.close()

for line in range(len(betaext_center)):
    betacenter_file.write(str(betaext_center[line]))
    betacenter_file.write("\n")
betacenter_file.close()
"""

## 3) Make histograms for guide fields, beta, and magnetization

### 3.1) Define some general parameters for plotting

In [ ]:
fontsize = 13
latexsize = 18
hfont = {'fontname':'Helvetica'}

mylabel_d = "disc"
mycolor_d = "blue"
mylabel_o = "outflows"
mycolor_o = "green"
mylabel_j = "jet"
mycolor_j = "red"

### 3.2) Plot disc vs outflows

In [ ]:
myfilename_bd = "/work/gustavo/gyst/Be105_smallSANE/beta_2D_disc.dat"
myfilename_bo = "/work/gustavo/gyst/Be105_smallSANE/beta_2D_outflows.dat"

myfile_bd = open(myfilename_bd, "r")
mydata_bd = pd.read_csv(myfilename_bd, header=None)
betaext_d = mydata_bd.values
#betaext_filt_d = betaext_d[~is_outlier(betaext_d)]
betaext_filt_d = betaext_d
betaext_filt_d = np.array(betaext_filt_d)
betaext_large_d = betaext_d[betaext_d >= 1]
betaext_large_d = np.array(betaext_large_d)
myfile_bd.close()

myfile_bo = open(myfilename_bo, "r")
mydata_bo = pd.read_csv(myfilename_bo, header=None)
betaext_o = mydata_bo.values
#betaext_filt_o = betaext_o[~is_outlier(betaext_o)]
betaext_filt_o = betaext_o
betaext_filt_o = np.array(betaext_filt_o)
betaext_large_o = betaext_o[betaext_o >= 1]
betaext_large_o = np.array(betaext_large_o)
myfile_bo.close()

#######################################################

myfilename_sd = "/work/gustavo/gyst/Be105_smallSANE/sigma_2D_disc.dat"
myfilename_so = "/work/gustavo/gyst/Be105_smallSANE/sigma_2D_outflows.dat"

myfile_sd = open(myfilename_sd, "r")
mydata_sd = pd.read_csv(myfilename_sd, header=None)
sigmaext_d = mydata_sd.values
#sigmaext_filt_d = sigmaext_d[~is_outlier(sigmaext_d)]
sigmaext_filt_d = sigmaext_d
sigmaext_filt_d = np.array(sigmaext_filt_d)
sigmaext_large_d = sigmaext_d[sigmaext_d >= 1]
sigmaext_large_d = np.array(sigmaext_large_d)
myfile_sd.close()

myfile_so = open(myfilename_so, "r")
mydata_so = pd.read_csv(myfilename_so, header=None)
sigmaext_o = mydata_so.values
#sigmaext_filt_o = sigmaext_o[~is_outlier(sigmaext_o)]
sigmaext_filt_o = sigmaext_o
sigmaext_filt_o = np.array(sigmaext_filt_o)
sigmaext_large_o = sigmaext_o[sigmaext_o >= 1]
sigmaext_large_o = np.array(sigmaext_large_o)
myfile_so.close()

##############################################################

myfilename_rd = "/work/gustavo/gyst/Be105_smallSANE/guidefield_r_2D_disc.dat"
myfilename_ro = "/work/gustavo/gyst/Be105_smallSANE/guidefield_r_2D_outflows.dat"

myfile_rd = open(myfilename_rd, "r")
mydata_rd = pd.read_csv(myfilename_rd, header=None)
BrBphiext_d = mydata_rd.values
#BrBphiext_filt_d = BrBphiext_d[~is_outlier(BthBphiext_d)]
BrBphiext_filt_d = BrBphiext_d
BrBphiext_filt_d = np.array(BrBphiext_filt_d)
BrBphiext_small_d = BrBphiext_d[BrBphiext_d < 1]
BrBphiext_small_d = np.array(BrBphiext_small_d)
myfile_rd.close()

myfile_rj = open(myfilename_rj, "r")
mydata_rj = pd.read_csv(myfilename_rj, header=None)
BrBphiext_j = mydata_rj.values
#BrBphiext_filt_j = BBphiext_j[~is_outlier(BthBphiext_j)]
BrBphiext_filt_j = BrBphiext_j
BrBphiext_filt_j = np.array(BrBphiext_filt_j)
BrBphiext_small_j = BrBphiext_j[BrBphiext_j < 1]
BrBphiext_small_j = np.array(BrBphiext_small_j)
myfile_rj.close()

################################

myfilename_td = "/work/gustavo/gyst/Be105_smallSANE/guidefield_th_2D_disc.dat"
myfilename_to = "/work/gustavo/gyst/Be105_smallSANE/guidefield_th_2D_outflows.dat"

myfile_td = open(myfilename_td, "r")
mydata_td = pd.read_csv(myfilename_td, header=None)
BthBphiext_d = mydata_td.values
#BthBphiext_filt_d = BthBphiext_d[~is_outlier(BthBphiext_d)]
BthBphiext_filt_d = BthBphiext_d
BthBphiext_filt_d = np.array(BthBphiext_filt_d)
BthBphiext_large_d = BthBphiext_d[BthBphiext_d >= 1]
BthBphiext_large_d = np.array(BthBphiext_large_d)
myfile_td.close()

myfile_to = open(myfilename_to, "r")
mydata_to = pd.read_csv(myfilename_to, header=None)
BthBphiext_o = mydata_to.values
#BthBphiext_filt_o = BthBphiext_o[~is_outlier(BthBphiext_o)]
BthBphiext_filt_o = BthBphiext_o
BthBphiext_filt_o = np.array(BthBphiext_filt_o)
BthBphiext_large_o = BthBphiext_j[BthBphiext_o >= 1]
BthBphiext_large_o = np.array(BthBphiext_large_o)
myfile_to.close()

##################################

fig = plt.figure(figsize=[6.4, 7.2])
#fig = plt.figure(figsize=[12.8, 14.4])
gs = GridSpec(2,2)

###########################################

ax1 = plt.subplot(gs[0,0])
plt.hist(np.log10(BrBphiext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(BrBphiext_filt_o), bins=20, rwidth=0.9, range=[-5,5], label="outflows", color=mycolor_o, zorder=2, alpha=0.6)
#plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log(B_r/B_\phi)$", fontsize=latexsize)
#plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

##################################################

ax2 = plt.subplot(gs[0,1])
plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(BthBphiext_filt_o), bins=20, rwidth=0.9, range=[-5,5], label="outflows", color=mycolor_o, zorder=2, alpha=0.6)
#plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log(B_\theta/B_\phi)$", fontsize=latexsize)
#plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

###############################################

ax3 = plt.subplot(gs[1,0])
plt.hist(np.log10(betaext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(betaext_filt_o), bins=20, rwidth=0.9, range=[-5,5], label="outflows", color=mycolor_o, zorder=2, alpha=0.6)
#plt.hist(np.log10(betaext_filt_d), bins=20, rwidth=0.9, range=[-5,10], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log\;\beta$", fontsize=latexsize)
plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

#############################################

ax4 = plt.subplot(gs[1,1])
plt.hist(np.log10(sigmaext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(sigmaext_filt_o), bins=20, rwidth=0.9, range=[-5,5], label="outflows", color=mycolor_o, zorder=2, alpha=0.6)
#plt.hist(np.log10(sigmaext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log\;\sigma$", fontsize=latexsize)
#plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

##################################################

savef=0
if (savef):
    plt.savefig("log_quantities_2D_discoutflows", dpi=150)

### 3.3) Plot disc vs jet

In [ ]:
myfilename_bd = "/work/gustavo/gyst/beta_2D_disc.dat"
myfilename_bj = "/work/gustavo/gyst/beta_2D_jet.dat"

myfile_bd = open(myfilename_bd, "r")
mydata_bd = pd.read_csv(myfilename_bd, header=None)
betaext_d = mydata_bd.values
#betaext_filt_d = betaext_d[~is_outlier(betaext_d)]
betaext_filt_d = betaext_d
betaext_filt_d = np.array(betaext_filt_d)
betaext_large_d = betaext_d[betaext_d >= 1]
betaext_large_d = np.array(betaext_large_d)
myfile_bd.close()

myfile_bj = open(myfilename_bj, "r")
mydata_bj = pd.read_csv(myfilename_bj, header=None)
betaext_j = mydata_bj.values
#betaext_filt_j = betaext_j[~is_outlier(betaext_j)]
betaext_filt_j = betaext_j
betaext_filt_j = np.array(betaext_filt_j)
betaext_large_j = betaext_j[betaext_j >= 1]
betaext_large_j = np.array(betaext_large_j)
myfile_bj.close()

#######################################################

myfilename_sd = "/work/gustavo/gyst/sigma_2D_disc.dat"
myfilename_sj = "/work/gustavo/gyst/sigma_2D_jet.dat"

myfile_sd = open(myfilename_sd, "r")
mydata_sd = pd.read_csv(myfilename_sd, header=None)
sigmaext_d = mydata_sd.values
#sigmaext_filt_d = sigmaext_d[~is_outlier(sigmaext_d)]
sigmaext_filt_d = sigmaext_d
sigmaext_filt_d = np.array(sigmaext_filt_d)
sigmaext_large_d = sigmaext_d[sigmaext_d >= 1]
sigmaext_large_d = np.array(sigmaext_large_d)
myfile_sd.close()

myfile_sj = open(myfilename_sj, "r")
mydata_sj = pd.read_csv(myfilename_sj, header=None)
sigmaext_j = mydata_sj.values
#sigmaext_filt_j = sigmaext_j[~is_outlier(sigmaext_j)]
sigmaext_filt_j = sigmaext_j
sigmaext_filt_j = np.array(sigmaext_filt_j)
sigmaext_large_j = sigmaext_j[sigmaext_j >= 1]
sigmaext_large_j = np.array(sigmaext_large_j)
myfile_sj.close()

##############################################################

myfilename_rd = "/work/gustavo/gyst/guidefield_r_2D_disc.dat"
myfilename_rj = "/work/gustavo/gyst/guidefield_r_2D_jet.dat"

myfile_rd = open(myfilename_rd, "r")
mydata_rd = pd.read_csv(myfilename_rd, header=None)
BrBphiext_d = mydata_rd.values
#BrBphiext_filt_d = BrBphiext_d[~is_outlier(BthBphiext_d)]
BrBphiext_filt_d = BrBphiext_d
BrBphiext_filt_d = np.array(BrBphiext_filt_d)
BrBphiext_small_d = BrBphiext_d[BrBphiext_d < 1]
BrBphiext_small_d = np.array(BrBphiext_small_d)
myfile_rd.close()

myfile_rj = open(myfilename_rj, "r")
mydata_rj = pd.read_csv(myfilename_rj, header=None)
BrBphiext_j = mydata_rj.values
#BrBphiext_filt_j = BBphiext_j[~is_outlier(BthBphiext_j)]
BrBphiext_filt_j = BrBphiext_j
BrBphiext_filt_j = np.array(BrBphiext_filt_j)
BrBphiext_small_j = BrBphiext_j[BrBphiext_j < 1]
BrBphiext_small_j = np.array(BrBphiext_small_j)
myfile_rj.close()

################################

myfilename_td = "/work/gustavo/gyst/guidefield_th_2D_disc.dat"
myfilename_tj = "/work/gustavo/gyst/guidefield_th_2D_jet.dat"

myfile_td = open(myfilename_td, "r")
mydata_td = pd.read_csv(myfilename_td, header=None)
BthBphiext_d = mydata_td.values
#BthBphiext_filt_d = BthBphiext_d[~is_outlier(BthBphiext_d)]
BthBphiext_filt_d = BthBphiext_d
BthBphiext_filt_d = np.array(BthBphiext_filt_d)
BthBphiext_large_d = BthBphiext_d[BthBphiext_d >= 1]
BthBphiext_large_d = np.array(BthBphiext_large_d)
myfile_td.close()

myfile_tj = open(myfilename_tj, "r")
mydata_tj = pd.read_csv(myfilename_tj, header=None)
BthBphiext_j = mydata_tj.values
#BthBphiext_filt_j = BthBphiext_j[~is_outlier(BthBphiext_j)]
BthBphiext_filt_j = BthBphiext_j
BthBphiext_filt_j = np.array(BthBphiext_filt_j)
BthBphiext_large_j = BthBphiext_j[BthBphiext_j >= 1]
BthBphiext_large_j = np.array(BthBphiext_large_j)
myfile_tj.close()

##################################

fig = plt.figure(figsize=[6.4, 7.2])
#fig = plt.figure(figsize=[12.8, 14.4])
gs = GridSpec(2,2)

###########################################

ax1 = plt.subplot(gs[0,0])
plt.hist(np.log10(BrBphiext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(BrBphiext_filt_j), bins=20, rwidth=0.9, range=[-5,5], label="jet", color=mycolor_j, zorder=2, alpha=0.6)
#plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log(B_r/B_\phi)$", fontsize=latexsize)
#plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

##################################################

ax2 = plt.subplot(gs[0,1])
plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(BthBphiext_filt_j), bins=20, rwidth=0.9, range=[-5,5], label="jet", color=mycolor_j, zorder=2, alpha=0.6)
#plt.hist(np.log10(BthBphiext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log(B_\theta/B_\phi)$", fontsize=latexsize)
#plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

###############################################

ax3 = plt.subplot(gs[1,0])
plt.hist(np.log10(betaext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(betaext_filt_j), bins=20, rwidth=0.9, range=[-5,5], label="jet", color=mycolor_j, zorder=2, alpha=0.6)
#plt.hist(np.log10(betaext_filt_d), bins=20, rwidth=0.9, range=[-5,10], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log\;\beta$", fontsize=latexsize)
plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

#############################################

ax4 = plt.subplot(gs[1,1])
plt.hist(np.log10(sigmaext_filt_d), bins=20, rwidth=0.9, range=[-5,5], label="disc", color=mycolor_d, zorder=2, alpha=0.6)
plt.hist(np.log10(sigmaext_filt_j), bins=20, rwidth=0.9, range=[-5,5], label="jet", color=mycolor_j, zorder=2, alpha=0.6)
#plt.hist(np.log10(sigmaext_filt_d), bins=20, rwidth=0.9, range=[-13,2], label="disc", color=mycolor_d)
_, max_ = plt.ylim()
plt.xlabel(r"$\log\;\sigma$", fontsize=latexsize)
#plt.ylabel("N", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5, zorder=1)
plt.grid(axis='y', alpha=0.5, zorder=1)
plt.legend(loc="best")
plt.xticks([-5,0,5],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.tight_layout()

##################################################

savef=0
if (savef):
    plt.savefig("log_quantities_2D_discjet", dpi=150)

### 3.4) Get mean, std deviation, and median values of guide fields, beta, and magnetization

In [ ]:
mean_BrBphi_d = np.mean(BrBphiext_filt_d)
std_BrBphi_d = np.std(BrBphiext_filt_d)
median_BrBphi_d = np.median(BrBphiext_filt_d)

mean_BthBphi_d = np.mean(BthBphiext_filt_d)
std_BthBphi_d = np.std(BthBphiext_filt_d)
median_BthBphi_d = np.median(BthBphiext_filt_d)

mean_beta_d = np.mean(betaext_filt_d)
std_beta_d = np.std(betaext_filt_d)
median_beta_d = np.median(betaext_filt_d)

mean_sigma_d = np.mean(sigmaext_filt_d)
std_sigma_d = np.std(sigmaext_filt_d)
median_sigma_d = np.median(sigmaext_filt_d)

mean_BrBphi_o = np.mean(BrBphiext_filt_o)
std_BrBphi_o = np.std(BrBphiext_filt_o)
median_BrBphi_o = np.median(BrBphiext_filt_o)

mean_BthBphi_o = np.mean(BthBphiext_filt_o)
std_BthBphi_o = np.std(BthBphiext_filt_o)
median_BthBphi_o = np.median(BthBphiext_filt_o)

mean_beta_o = np.mean(betaext_filt_o)
std_beta_o = np.std(betaext_filt_o)
median_beta_o = np.median(betaext_filt_o)

mean_sigma_o = np.mean(sigmaext_filt_o)
std_sigma_o = np.std(sigmaext_filt_o)
median_sigma_o = np.median(sigmaext_filt_o)

mean_BrBphi_j = np.mean(BrBphiext_filt_j)
std_BrBphi_j = np.std(BrBphiext_filt_j)
median_BrBphi_j = np.median(BrBphiext_filt_j)

mean_BthBphi_j = np.mean(BthBphiext_filt_j)
std_BthBphi_j = np.std(BthBphiext_filt_j)
median_BthBphi_j = np.median(BthBphiext_filt_j)

mean_beta_j = np.mean(betaext_filt_j)
std_beta_j = np.std(betaext_filt_j)
median_beta_j = np.median(betaext_filt_j)

mean_sigma_j = np.mean(sigmaext_filt_j)
std_sigma_j = np.std(sigmaext_filt_j)
median_sigma_j = np.median(sigmaext_filt_j)

### 3.5) Print mean, std dev and median for these 4 quantities

In [1]:
print("DISC")
print("MEAN, STD DEV, MEDIAN")
print("BETA: ", mean_beta_d, std_beta_d, median_beta_d)
print("SIGMA: ", mean_sigma_d, std_sigma_d, median_sigma_d)
print("Br/Bphi: ", mean_BrBphi_d, std_BrBphi_d, median_BrBphi_d)
print("Bth/Bphi: ", mean_BthBphi_d, std_BthBphi_d, median_BthBphi_d)

print("OUTFLOWS")
print("MEAN, STD DEV, MEDIAN")
print("BETA: ", mean_beta_o, std_beta_o, median_beta_o)
print("SIGMA: ", mean_sigma_o, std_sigma_o, median_sigma_o)
print("Br/Bphi: ", mean_BrBphi_o, std_BrBphi_o, median_BrBphi_o)
print("Bth/Bphi: ", mean_BthBphi_o, std_BthBphi_o, median_BthBphi_o)

print("JET")
print("MEAN, STD DEV, MEDIAN")
print("BETA: ", mean_beta_j, std_beta_j, median_beta_j)
print("SIGMA: ", mean_sigma_j, std_sigma_j, median_sigma_j)
print("Br/Bphi: ", mean_BrBphi_j, std_BrBphi_j, median_BrBphi_j)
print("Bth/Bphi: ", mean_BthBphi_j, std_BthBphi_j, median_BthBphi_j)

MEAN, STD DEV, MEDIAN
